# S4E5 Regression with a Flood Prediction Dataset

## Introduction

This notebook is aimed to perform regression analysis on a flood prediction dataset. The evaluation metric is $R^2$.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import optuna

DATA_PATH = './data/'
ORIGINAL_DATA_PATH = DATA_PATH
SUBMISSIONS_PATH = './submissions/'
TEMP_PATH = './temp/'

# Load the data
train = pd.read_csv(DATA_PATH + 'train.csv', index_col='id')
test = pd.read_csv(DATA_PATH + 'test.csv', index_col='id')
original = pd.read_csv(ORIGINAL_DATA_PATH + 'flood.csv')
original.index.rename('id', inplace=True)
new_train = pd.concat([train, original], axis=0)

## Exploratory Data Analysis

### First glance of the data

In [33]:
train.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
id,,,,,,,,,,,,,,,,,,,,,
0,5,8,5,8,6,4,4,3,3,4,...,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,...,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,...,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,...,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [34]:
original.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
id,,,,,,,,,,,,,,,,,,,,,
0,3,8,6,6,4,4,6,2,3,2,...,10,7,4,2,3,4,3,2,6,0.450
1,8,4,5,7,7,9,1,5,5,4,...,9,2,6,2,1,1,9,1,3,0.475
2,3,10,4,1,7,5,4,7,4,9,...,7,4,4,8,6,1,8,3,6,0.515
3,4,4,2,7,3,4,1,4,6,4,...,4,2,6,6,8,8,6,6,10,0.520
4,3,7,5,2,5,8,5,2,7,5,...,7,6,5,3,3,4,4,3,4,0.475


In [35]:
train.describe()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
count,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,...,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06,1.117957e+06
mean,4.921450e+00,4.926671e+00,4.955322e+00,4.942240e+00,4.942517e+00,4.934093e+00,4.955878e+00,4.927791e+00,4.942619e+00,4.949230e+00,...,4.946893e+00,4.953999e+00,4.931376e+00,4.929032e+00,4.925907e+00,4.927520e+00,4.950859e+00,4.940587e+00,4.939004e+00,5.044803e-01
std,2.056387e+00,2.093879e+00,2.072186e+00,2.051689e+00,2.083391e+00,2.057742e+00,2.083063e+00,2.065992e+00,2.068545e+00,2.083324e+00,...,2.072333e+00,2.088899e+00,2.078287e+00,2.082395e+00,2.064813e+00,2.074176e+00,2.068696e+00,2.081123e+00,2.090350e+00,5.102610e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.850000e-01
25%,3.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,...,4.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,4.700000e-01
50%,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,...,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.050000e-01
75%,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,...,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,5.400000e-01
max,1.600000e+01,1.800000e+01,1.600000e+01,1.700000e+01,1.700000e+01,1.700000e+01,1.600000e+01,1.600000e+01,1.600000e+01,1.800000e+01,...,1.700000e+01,1.700000e+01,1.600000e+01,1.600000e+01,1.700000e+01,1.800000e+01,1.900000e+01,1.600000e+01,1.600000e+01,7.250000e-01


In [36]:
original.describe()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
count,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000
mean,4.991480,4.984100,5.01594,5.008480,4.989060,4.988340,5.01536,4.988600,5.006120,5.006380,...,5.006060,4.999920,4.984220,4.97982,4.988200,4.984980,5.00512,4.994360,4.990520,0.499660
std,2.236834,2.246488,2.23131,2.222743,2.243159,2.226761,2.24500,2.232642,2.234588,2.241633,...,2.238107,2.247101,2.227741,2.23219,2.231134,2.238279,2.23176,2.230011,2.246075,0.050034
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.285000
25%,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000,3.00000,3.000000,3.000000,0.465000
50%,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.00000,5.000000,5.000000,0.500000
75%,6.000000,6.000000,6.00000,6.000000,6.000000,6.000000,6.00000,6.000000,6.000000,6.000000,...,6.000000,6.000000,6.000000,6.00000,6.000000,6.000000,6.00000,6.000000,6.000000,0.535000
max,16.000000,18.000000,16.00000,17.000000,17.000000,17.000000,16.00000,16.000000,16.000000,18.000000,...,17.000000,17.000000,16.000000,16.00000,17.000000,19.000000,22.00000,16.000000,16.000000,0.725000


### Check for missing values

In [37]:
print("Training set")
print(train.isnull().sum())
print("Test set")
print(test.isnull().sum())
print("Original set")
print(original.isnull().sum())

# we don't have any missing values.

Training set
MonsoonIntensity                   0
TopographyDrainage                 0
RiverManagement                    0
Deforestation                      0
Urbanization                       0
ClimateChange                      0
DamsQuality                        0
Siltation                          0
AgriculturalPractices              0
Encroachments                      0
IneffectiveDisasterPreparedness    0
DrainageSystems                    0
CoastalVulnerability               0
Landslides                         0
Watersheds                         0
DeterioratingInfrastructure        0
PopulationScore                    0
WetlandLoss                        0
InadequatePlanning                 0
PoliticalFactors                   0
FloodProbability                   0
dtype: int64
Test set
MonsoonIntensity                   0
TopographyDrainage                 0
RiverManagement                    0
Deforestation                      0
Urbanization                       0
Cli

### Check distributions from three datasets

In [38]:
features = train.drop('FloodProbability', axis=1).columns


# plt.figure(figsize=(20, 20))
# n_rows = 5
# n_cols = 4

# for i, feature in enumerate(features, 1):
#     plt.subplot(n_rows, n_cols, i)
#     sns.histplot(train[feature], stat="density", color="blue", label="Train", kde=False, element="step", fill=False)
#     sns.histplot(test[feature], stat="density", color="green", label="Test", kde=False, element="step", fill=False)
#     sns.histplot(original[feature], stat="density", color="red", label="Original", kde=False, element="step", fill=False)
    
#     plt.title(f'Histogram of {feature}')
#     plt.xlabel(feature)
#     plt.ylabel('Density')
#     if i == 1:  # Only show legend in the first subplot for clarity
#         plt.legend()

# # Adjust layout
# plt.tight_layout()
# plt.show()


# everything looks fine, the distributions are similar.
# almost all distribution looks like normal distribution that is skewed to the right.

In [39]:
# plot the y values
# plt.figure(figsize=(10, 5))
# sns.histplot(train['FloodProbability'], stat="density", color="blue", label="Train", kde=False, element="step")
# sns.histplot(original['FloodProbability'], stat="density", color="red", label="Original", kde=False, element="step")
# plt.title('Histogram of FloodProbability')
# plt.xlabel('FloodProbability')
# plt.ylabel('Density')
# plt.legend()
# plt.show()


### Check the correlation

In [40]:
# calculate the correlation between the features and the target
correlation = train.corrwith(train['FloodProbability']).sort_values(ascending=False)
correlation

FloodProbability                   1.000000
DeterioratingInfrastructure        0.190007
MonsoonIntensity                   0.189098
DamsQuality                        0.187996
TopographyDrainage                 0.187635
RiverManagement                    0.187131
Siltation                          0.186789
PopulationScore                    0.185890
Landslides                         0.185346
ClimateChange                      0.184761
Deforestation                      0.184001
WetlandLoss                        0.183396
AgriculturalPractices              0.183366
IneffectiveDisasterPreparedness    0.183109
PoliticalFactors                   0.182417
Watersheds                         0.181907
InadequatePlanning                 0.180968
Urbanization                       0.180861
DrainageSystems                    0.179305
Encroachments                      0.178841
CoastalVulnerability               0.177774
dtype: float64

## Baseline score with a simple linear regression model

In [3]:
# use 5-fold cv and r2 score to evaluate a simple linear regression model
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, r2_score

X = new_train.drop('FloodProbability', axis=1)
y = new_train['FloodProbability']

# model = LinearRegression()
# cv_score = cross_val_score(model, X, y, cv=5, scoring=make_scorer(r2_score))
# print(cv_score.mean())

In [42]:
# use 5-fold cv and r2 score to evaluate a LightGBM regression model
# from lightgbm import LGBMRegressor

# model = LGBMRegressor(random_state=0)
# cv_score = cross_val_score(model, X, y, cv=5, scoring=make_scorer(r2_score))
# print(cv_score.mean())

In [43]:
# use 5-fold cv and r2 score to evaluate a XGBoost regression model
# from xgboost import XGBRegressor

# model = XGBRegressor(random_state=0, device='cuda')
# cv_score = cross_val_score(model, X, y, cv=5, scoring=make_scorer(r2_score))
# print(cv_score.mean())

In [44]:
# use 5-fold cv and r2 score to evaluate a CatBoost regression model
# from catboost import CatBoostRegressor

# model = CatBoostRegressor(random_state=0, task_type="GPU", devices='0')
# cv_score = cross_val_score(model, X, y, cv=5, scoring=make_scorer(r2_score))
# print(cv_score.mean())

In [45]:
# train a autogluon regression model
# from autogluon.tabular import TabularPredictor

# predictor = TabularPredictor(label='FloodProbability', path=MODELS_PATH).fit(train, ag_args_fit={'num_gpus': 1})

# y_preds = predictor.predict(test)
# submission = pd.DataFrame({'FloodProbability': y_preds}, index=test.index)
# submission.to_csv(SUBMISSIONS_PATH + 'autogluon.csv')

## Feature Engineering

In [2]:
NON_FEATURES = ['FloodProbability', 'fold']
BASE_FEATURES = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']

# meta features from https://www.kaggle.com/code/igorvolianiuk/flood-prediction-lgbm
def add_features(df):
    df['total'] = df[BASE_FEATURES].sum(axis=1)
    df['amplified_sum'] = (df[BASE_FEATURES] ** 1.5).sum(axis=1)
    df['fskew'] = df[BASE_FEATURES].skew(axis=1)
    df['fkurtosis'] = df[BASE_FEATURES].kurtosis(axis=1)
    df['mean'] = df[BASE_FEATURES].mean(axis=1)
    df['std'] = df[BASE_FEATURES].std(axis=1)
    df['max'] = df[BASE_FEATURES].max(axis=1)
    df['min'] = df[BASE_FEATURES].min(axis=1)
    df['range'] = df['max'] - df['min']
    df['median'] = df[BASE_FEATURES].median(axis=1)
    df['ptp'] = df[BASE_FEATURES].values.ptp(axis=1)
    df['q25'] = df[BASE_FEATURES].quantile(0.25, axis=1)
    df['q75'] = df[BASE_FEATURES].quantile(0.75, axis=1)
    return df

new_train = add_features(new_train)
test = add_features(test)

In [3]:
# save the new dataset

new_train.to_csv(DATA_PATH + "new_train.csv", index=True)
test.to_csv(DATA_PATH + "new_test.csv", index=True)

## Hyperparameter tuning

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X = new_train.drop('FloodProbability', axis=1)
y = new_train['FloodProbability']

X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X, y, test_size=0.2, random_state=0)
X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(X, y, test_size=0.2, random_state=1)

scaler_1 = MinMaxScaler()
X_train_1 = scaler_1.fit_transform(X_train_1)
X_val_1 = scaler_1.transform(X_val_1)

scaler_2 = MinMaxScaler()
X_train_2 = scaler_2.fit_transform(X_train_2)
X_val_2 = scaler_2.transform(X_val_2)

### Tuning Lasso regression

In [48]:
from sklearn.linear_model import Lasso

def objective(trial):
    # Suggest values for the hyperparameters
    alpha = trial.suggest_float('alpha', 1e-10, 1, log=True)  # Regularization strength, typical range for Lasso
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    max_iter = trial.suggest_int('max_iter', 100, 1500)  # Number of iterations
    
    # Create the Lasso regression model with suggested hyperparameters
    model = Lasso(alpha=alpha, fit_intercept=fit_intercept, max_iter=max_iter, random_state=42)
    
    model.fit(X_train_1, y_train_1)
    pred_1 = model.predict(X_val_1)
    score_1 = r2_score(y_val_1, pred_1)

    model.fit(X_train_2, y_train_2)
    pred_2 = model.predict(X_val_2)
    score_2 = r2_score(y_val_2, pred_2)

    score = (score_1 + score_2) / 2
    if np.isnan(score):
        score = -100
    return score

# # Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-05-06 21:24:02,195] A new study created in memory with name: no-name-931a500e-3e08-462d-a777-003ff3229cf9
[I 2024-05-06 21:24:06,168] Trial 0 finished with value: 0.8487567135180376 and parameters: {'alpha': 4.766065981776375e-09, 'fit_intercept': True, 'max_iter': 437}. Best is trial 0 with value: 0.8487567135180376.
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.926e+02, tolerance: 2.400e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.924e+02, tolerance: 2.400e+01
  model = cd_fast

In [ ]:
# Results
print("Best trial:")
trial = study.best_trial
print("Value of R2:", trial.value)
print("Best hyperparameters:")
for key, value in trial.params.items():
    print(f" {key}: {value}")
    
# Best trial:
#  Value of R2: 0.8484212349618604
#  Best hyperparameters:
#  alpha: 1.321069293763322e-05
#  fit_intercept: True
#  max_iter: 1289

Best trial:
 Value of R2: 0.8484212349618604
 Best hyperparameters:
 alpha: 1.321069293763322e-05
 fit_intercept: True
 max_iter: 1289


### Ridge regression

In [7]:
from sklearn.linear_model import Ridge

def objective(trial):
    # Suggest values for the hyperparameters
    alpha = trial.suggest_float('alpha', 1e-10, 1e10, log=True)  # Regularization strength
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    max_iter = trial.suggest_int('max_iter', 100, 1000)  # Number of iterations
    
    # Create the Ridge regression model with suggested hyperparameters
    model = Ridge(alpha=alpha, fit_intercept=fit_intercept, max_iter=max_iter, random_state=42)
    
    model.fit(X_train_1, y_train_1)
    pred_1 = model.predict(X_val_1)
    score_1 = r2_score(y_val_1, pred_1)

    model.fit(X_train_2, y_train_2)
    pred_2 = model.predict(X_val_2)
    score_2 = r2_score(y_val_2, pred_2)

    score = (score_1 + score_2) / 2
    if np.isnan(score):
        score = -100
    return score

# # Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-05-07 17:41:19,238] A new study created in memory with name: no-name-29bba7ed-9f4a-4366-a55c-5a1d6b5f4a48
[I 2024-05-07 17:41:19,907] Trial 0 finished with value: 0.5604784960059348 and parameters: {'alpha': 74281.5465793234, 'fit_intercept': True, 'max_iter': 863}. Best is trial 0 with value: 0.5604784960059348.
[I 2024-05-07 17:41:20,319] Trial 1 finished with value: 0.849162046484518 and parameters: {'alpha': 0.00044171326922667306, 'fit_intercept': False, 'max_iter': 599}. Best is trial 1 with value: 0.849162046484518.
[I 2024-05-07 17:41:20,873] Trial 2 finished with value: 0.8491620468707313 and parameters: {'alpha': 9.648094822769735e-10, 'fit_intercept': True, 'max_iter': 127}. Best is trial 2 with value: 0.8491620468707313.
[I 2024-05-07 17:41:21,258] Trial 3 finished with value: 0.848938848513006 and parameters: {'alpha': 71.86678318292836, 'fit_intercept': False, 'max_iter': 137}. Best is trial 2 with value: 0.8491620468707313.
[I 2024-05-07 17:41:21,667] Trial 4 fin

KeyboardInterrupt: 

In [ ]:
# Results
print("Best trial:")
trial = study.best_trial
print("Value of R2:", trial.value)
print("Best hyperparameters:")
for key, value in trial.params.items():
    print(f" {key}: {value}")

# Best trial:
#  Value of R2: 0.8484213857177292
#  Best hyperparameters:
#  alpha: 2550.842624233109
#  fit_intercept: True
#  max_iter: 677

Best trial:
 Value of R2: 0.8484213857177292
 Best hyperparameters:
 alpha: 2550.842624233109
 fit_intercept: True
 max_iter: 677


### LightGBM Regressor

In [6]:
from lightgbm import LGBMRegressor

def objective(trial):
    # Suggest values for various hyperparameters
    param = {
        'objective': 'regression',
        'verbosity': -1,
        'device': 'cuda',
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 400000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10.0, log=True)
    }
   
    try:
    # Create and train LightGBM regressor model
        model = LGBMRegressor(**param, random_state=42)
        
        model.fit(X_train_1, y_train_1)
        pred_1 = model.predict(X_val_1)
        score_1 = r2_score(y_val_1, pred_1)

        model.fit(X_train_2, y_train_2)
        pred_2 = model.predict(X_val_2)
        score_2 = r2_score(y_val_2, pred_2)
    except:
        return -100
    score = (score_1 + score_2) / 2
    
    if np.isnan(score):
        print(score)
        score = -100
    return score

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2024-05-06 21:28:49,065] A new study created in memory with name: no-name-317ee820-1ddd-40e5-9cb8-5a86d1cfc375
[I 2024-05-06 21:29:48,474] Trial 0 finished with value: -8.495336493939572 and parameters: {'boosting_type': 'dart', 'num_leaves': 219, 'subsample_for_bin': 283642, 'min_child_samples': 453, 'max_depth': 4, 'learning_rate': 0.0050118375321069894, 'n_estimators': 897, 'subsample': 0.8089145876715045, 'colsample_bytree': 0.570319752065839, 'reg_alpha': 0.0781070630095452, 'reg_lambda': 0.007737577616687532}. Best is trial 0 with value: -8.495336493939572.
[I 2024-05-06 21:32:15,581] Trial 1 finished with value: 0.8660633524983765 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 134, 'subsample_for_bin': 259638, 'min_child_samples': 450, 'max_depth': 18, 'learning_rate': 0.004152159082671395, 'n_estimators': 623, 'subsample': 0.4650143582034575, 'colsample_bytree': 0.8479156083797528, 'reg_alpha': 0.00036638541106907304, 'reg_lambda': 0.0021088500661929897}. Best is tr

In [ ]:
# Results
print("Best trial:")
lgbm_best_trial = study.best_trial
print("Value of R2:", lgbm_best_trial.value)
print("Best hyperparameters:")
for key, value in lgbm_best_trial.params.items():
    print(f" {key}: {value}")

# Best trial:
#  Value of R2: 0.8377810276521979
#  Best hyperparameters:
#  num_leaves: 30
#  max_depth: 18
#  min_data_in_leaf: 24
#  learning_rate: 0.05603164191140428
#  n_estimators: 488
#  subsample: 0.838658725731085
#  colsample_bytree: 0.7943480266514407
#  reg_alpha: 0.44472632944161145
#  reg_lambda: 0.42385330246256925
#  min_gain_to_split: 0.002900266904476601
#  feature_fraction: 0.9750989139491191
#  bagging_fraction: 0.8914508401674793
#  bagging_freq: 4

lgbm_params ={
    'lambda': 5.878052368111836, 
    'alpha': 0.021036799354105035, 
    'subsample': 0.6708960856454209, 
    'colsample_bytree': 0.9689249019173221, 
    'max_depth': 9, 
    'min_child_weight': 8, 
    'eta': 0.010531200672103776, 
    'grow_policy': 'lossguide', 
    'n_estimators': 962
    }

# lgbm_params = {
#     'boosting_type': 'gbdt', 
#     'num_leaves': 227, 
#     'subsample_for_bin': 204195, 
#     'min_child_samples': 98, 
#     'max_depth': 14, 
#     'learning_rate': 0.008725580097840743, 
#     'n_estimators': 1486, 
#     'subsample': 0.6924206162743796, 
#     'colsample_bytree': 0.608985636026134, 
#     'reg_alpha': 0.000982304606619489, 
#     'reg_lambda': 4.733782716082672
#     }

### XGBoost regressor

In [6]:
from xgboost import XGBRegressor
import cupy
mempool = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()

def objective(trial):
    # Suggest values for various hyperparameters
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'eta': trial.suggest_float('eta', 0.01, 0.3, log=True),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),  # Include n_estimators in tuning
        'tree_method': 'gpu_hist',  # Use GPU acceleration
        'predictor': 'gpu_predictor',
        'device': 'cuda'
    }
    
    # Create and train XGBoost regressor model
    model = XGBRegressor(**param, random_state=42)
    
    model.fit(X_train_1, y_train_1)
    pred_1 = model.predict(X_val_1)
    score_1 = r2_score(y_val_1, pred_1)

    model.fit(X_train_2, y_train_2)
    pred_2 = model.predict(X_val_2)
    score_2 = r2_score(y_val_2, pred_2)

    score = (score_1 + score_2) / 2
    if np.isnan(score):
        score = -100
    return score
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

mempool.free_all_blocks()
pinned_mempool.free_all_blocks()

[I 2024-05-07 17:21:54,222] A new study created in memory with name: no-name-c56939c3-008c-43a9-958d-05c5b6e7f84e
[I 2024-05-07 17:22:33,588] Trial 0 finished with value: 0.868485866724964 and parameters: {'lambda': 0.0017182681140780257, 'alpha': 0.036407949751770544, 'subsample': 0.9459321607080899, 'colsample_bytree': 0.9822451655777837, 'max_depth': 9, 'min_child_weight': 8, 'eta': 0.11714541113497118, 'grow_policy': 'depthwise', 'n_estimators': 696}. Best is trial 0 with value: 0.868485866724964.
[I 2024-05-07 17:25:08,902] Trial 1 finished with value: 0.8648513716148123 and parameters: {'lambda': 0.0022950677987954157, 'alpha': 0.039440479682750974, 'subsample': 0.6859914664023445, 'colsample_bytree': 0.8517090216762131, 'max_depth': 6, 'min_child_weight': 5, 'eta': 0.1808012562408326, 'grow_policy': 'lossguide', 'n_estimators': 2942}. Best is trial 0 with value: 0.868485866724964.
[I 2024-05-07 17:25:49,696] Trial 2 finished with value: 0.8710390539252986 and parameters: {'lambd

KeyboardInterrupt: 

In [ ]:
# Results
print("Best trial:")
xgb_best_trial = study.best_trial
print("Value of R2:", xgb_best_trial.value)
print("Best hyperparameters:")
for key, value in xgb_best_trial.params.items():
    print(f" {key}: {value}")

xgboost_params = {
    'lambda': 0.05783196300702904, 
    'alpha': 0.0013359911973639838, 
    'subsample': 0.5555995831720429, 
    'colsample_bytree': 0.834809526216768, 
    'max_depth': 5, 
    'min_child_weight': 10, 
    'eta': 0.014058571266825881, 
    'grow_policy': 'lossguide', 
    'n_estimators': 1499
    }


### CatBoost Regressor

In [5]:
from catboost import CatBoostRegressor

def objective(trial):
    # Suggest values for various hyperparameters
    param = {
        'iterations': trial.suggest_int('iterations', 100, 3000),
        'depth': trial.suggest_int('depth', 1, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'random_strength': trial.suggest_int('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'task_type': 'GPU',  # Enable GPU acceleration
        'loss_function': 'RMSE',
        'verbose': False
    }
    
    # Create and train CatBoost regressor model
    model = CatBoostRegressor(**param, random_seed=42)
    
    model.fit(X_train_1, y_train_1)
    pred_1 = model.predict(X_val_1)
    score_1 = r2_score(y_val_1, pred_1)

    model.fit(X_train_2, y_train_2)
    pred_2 = model.predict(X_val_2)
    score_2 = r2_score(y_val_2, pred_2)

    score = (score_1 + score_2) / 2
    if np.isnan(score):
        score = -100
    return score

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-05-07 16:31:31,086] A new study created in memory with name: no-name-7a6ce3eb-258b-458f-aa72-257cb4b7e9b7
[I 2024-05-07 16:32:39,367] Trial 0 finished with value: 0.8700552007837836 and parameters: {'iterations': 1532, 'depth': 8, 'learning_rate': 0.15080159209721974, 'random_strength': 7, 'bagging_temperature': 0.857680489194646, 'l2_leaf_reg': 0.002834674422925088, 'border_count': 216}. Best is trial 0 with value: 0.8700552007837836.
[I 2024-05-07 16:33:16,526] Trial 1 finished with value: 0.8710824667159165 and parameters: {'iterations': 1418, 'depth': 5, 'learning_rate': 0.06607799100944327, 'random_strength': 3, 'bagging_temperature': 0.3580280105422403, 'l2_leaf_reg': 0.09659653488193709, 'border_count': 111}. Best is trial 1 with value: 0.8710824667159165.
[I 2024-05-07 16:33:25,328] Trial 2 finished with value: 0.870064317615322 and parameters: {'iterations': 304, 'depth': 5, 'learning_rate': 0.07842426978119739, 'random_strength': 20, 'bagging_temperature': 0.830046570

KeyboardInterrupt: 

In [ ]:
# # Results
print("Best trial:")
catboost_best_trial = study.best_trial
print("Value of R2:", catboost_best_trial.value)
print("Best hyperparameters:")
for key, value in catboost_best_trial.params.items():
    print(f" {key}: {value}")


catboost_params = {
    'iterations': 2976, 
    'depth': 8, 
    'learning_rate': 0.023410110911231772, 
    'random_strength': 11, 
    'bagging_temperature': 0.01652690262850942, 
    'l2_leaf_reg': 2.173016746274142, 
    'border_count': 198
    }


## Create ensemble model

In [8]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler

lgbm_params = {
    'lambda': 2.3188326473128194, 
    'alpha': 0.0019463147382006896, 
    'subsample': 0.6927717825949194, 
    'colsample_bytree': 0.8454809810463898, 
    'max_depth': 9, 
    'min_child_weight': 10, 
    'eta': 0.012177737027772991, 
    'grow_policy': 'lossguide', 
    'n_estimators': 1000
    }

xgboost_params = {
    'lambda': 2.3188326473128194, 
    'alpha': 0.0019463147382006896, 
    'subsample': 0.6927717825949194, 
    'colsample_bytree': 0.8454809810463898, 
    'max_depth': 9, 
    'min_child_weight': 10, 
    'eta': 0.012177737027772991, 
    'grow_policy': 'lossguide', 
    'n_estimators': 1000
    }

catboost_params = {
    'iterations': 2976, 
    'depth': 8, 
    'learning_rate': 0.023410110911231772, 
    'random_strength': 11, 
    'bagging_temperature': 0.01652690262850942, 
    'l2_leaf_reg': 2.173016746274142, 
    'border_count': 198
    }


lgbm = LGBMRegressor(**lgbm_params, objective='regression', verbosity=-1, device='cuda')
xgb = XGBRegressor(**xgboost_params, objective='reg:squarederror', booster='gbtree', tree_method='gpu_hist', predictor='gpu_predictor', verbosity=0, random_state=42)
catboost = CatBoostRegressor(**catboost_params, task_type='GPU', loss_function='RMSE', verbose=False, random_seed=42)


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and predict with each model
lgbm.fit(X_train, y_train)
xgb.fit(X_train, y_train)
catboost.fit(X_train, y_train)


pred_lgbm = lgbm.predict(X_val)
tmp = pd.DataFrame({'id': X_val.index, 'FloodProbability': pred_lgbm})
tmp.to_csv(TEMP_PATH + 'lgbm_val.csv', index=False)
pred_xgb = xgb.predict(X_val)
tmp = pd.DataFrame({'id': X_val.index, 'FloodProbability': pred_xgb})
tmp.to_csv(TEMP_PATH + 'xgb_val.csv', index=False)
pred_catboost = catboost.predict(X_val)
tmp = pd.DataFrame({'id': X_val.index, 'FloodProbability': pred_catboost})
tmp.to_csv(TEMP_PATH + 'catboost_val.csv', index=False)

In [9]:
def objective(trial):
    w_lgbm = trial.suggest_float('w_lgbm', 0, 1)
    w_xgb = trial.suggest_float('w_xgb', 0, 1)
    w_catboost = trial.suggest_float('w_catboost', 0, 1)

    
    # Normalize weights
    total = w_lgbm + w_xgb + w_catboost
    w_lgbm, w_xgb, w_catboost = [w / total for w in [w_lgbm, w_xgb, w_catboost]]
    
    # Ensemble prediction
    ensemble_pred = (w_lgbm * pred_lgbm + w_xgb * pred_xgb + w_catboost * pred_catboost)
    
    # Evaluate
    return r2_score(y_val, ensemble_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

print("Best trial:")
trial = study.best_trial
print(" Value of R2:", trial.value)
print(" Best hyperparameters:")
for key, value in trial.params.items():
    print(f" {key}: {value}")
    


[I 2024-05-07 21:03:05,481] A new study created in memory with name: no-name-401ccf0c-6e91-476a-a148-e1ee2bebdb82
[I 2024-05-07 21:03:05,494] Trial 0 finished with value: 0.8702210819200591 and parameters: {'w_lgbm': 0.594739419203231, 'w_xgb': 0.7077158051319679, 'w_catboost': 0.5480992216686414}. Best is trial 0 with value: 0.8702210819200591.
[I 2024-05-07 21:03:05,505] Trial 1 finished with value: 0.8702322357778358 and parameters: {'w_lgbm': 0.7571106291506108, 'w_xgb': 0.4107161308353722, 'w_catboost': 0.6993794687920131}. Best is trial 1 with value: 0.8702322357778358.
[I 2024-05-07 21:03:05,514] Trial 2 finished with value: 0.8702367113749921 and parameters: {'w_lgbm': 0.26818487526025625, 'w_xgb': 0.23503513247753227, 'w_catboost': 0.29819577602659864}. Best is trial 2 with value: 0.8702367113749921.
[I 2024-05-07 21:03:05,525] Trial 3 finished with value: 0.8701564417444467 and parameters: {'w_lgbm': 0.8206956367795615, 'w_xgb': 0.37823989997483853, 'w_catboost': 0.2124270489

Best trial:
 Value of R2: 0.8702732038552832
 Best hyperparameters:
 w_lgbm: 0.268539528806283
 w_xgb: 0.05807467683210701
 w_catboost: 0.9601739006621279


In [10]:


lgbm_model = LGBMRegressor(**lgbm_params, objective='regression', verbosity=-1, device='cuda')
xgb_model = XGBRegressor(**xgboost_params, objective='reg:squarederror', booster='gbtree', tree_method='gpu_hist', predictor='gpu_predictor', verbosity=0, random_state=42)
catboost_model = CatBoostRegressor(**catboost_params, task_type='GPU', loss_function='RMSE', verbose=False, random_seed=42)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(test)


lgbm_model.fit(X, y)
xgb_model.fit(X, y)
catboost_model.fit(X, y)

lgbm_preds = lgbm_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)
catboost_preds = catboost_model.predict(X_test)

ensemble_preds = (trial.params['w_lgbm'] * lgbm_preds + trial.params['w_xgb'] * xgb_preds + trial.params['w_catboost'] * catboost_preds) / sum(trial.params.values())

submission = pd.DataFrame({'FloodProbability': ensemble_preds}, index=test.index)
submission.to_csv(SUBMISSIONS_PATH + 'ensemble2.csv')